# `Convolution`

In [2]:
import numpy as np
from numpy.typing import NDArray
from typing import TypeVar, Sequence

F = TypeVar("F", float, np.floating)

def _convolve(image: NDArray[F], kernel: NDArray[F]) -> NDArray[F]:
    """
    Performs a 2D convolution between an image and a kernel.

    Parameters:
    -----------
    image : NDArray[F]
        A 2D NumPy array representing the grayscale image.
    kernel : NDArray[F]
        A 2D NumPy array representing the convolution kernel (filter).

    Returns:
    --------
    NDArray[F]
        The filtered image after applying the convolution.

    Notes:
    ------
    - Uses a **zero-padding** approach to maintain image dimensions.
    - Supports **grayscale images** only (single-channel).
    - Performs **valid convolution**, not cross-correlation.
    - Uses an element-wise multiplication of the window and kernel.

    Complexity:
    -----------
    - O(n * m * k^2), where (n, m) are the image dimensions and (k, k) is the kernel size.
    """

    # Get dimensions
    image_y, image_x = image.shape
    kernel_y, kernel_x = kernel.shape

    # Compute padding sizes based on kernel dimensions
    pad_y, pad_x = kernel_y // 2, kernel_x // 2

    # Initialize output image
    filtered_image = np.zeros_like(image, dtype = np.float32)

    # Apply zero-padding to the image
    padded_image = np.pad(image, pad_width = ((pad_y, pad_y), (pad_x, pad_x)), mode = 'constant', constant_values = 0)

    # Perform Convolution
    for y in range(image_y):
        for x in range(image_x):
            # Extract sliding window
            window = padded_image[y : y + kernel_y, x : x + kernel_x]
            # Apply convolution (dot product of window and kernel)
            filtered_image[y, x] = np.sum(window * kernel)

    return filtered_image


if __name__ == "__main__":
    # Create a sample 5x5 grayscale image
    image = np.random.rand(5, 5).astype(np.float32)

    # Define a basic edge detection kernel
    kernel = np.array([
        [0, -1,  0],
        [-1, 4, -1],
        [0, -1,  0]
    ], dtype = np.float32)

    # Apply convolution
    filtered_image = _convolve(image, kernel)
    
    print("Filtered Image:\n", filtered_image)



Filtered Image:
 [[ 1.7203203  -0.47367704  1.515836    0.77846134 -0.38133192]
 [-0.89293116  0.973206   -1.0218678  -0.51310873  0.49363786]
 [ 1.0410088   1.3334421  -0.23257571 -0.7441623   1.728864  ]
 [-1.3862215  -0.25389135  2.0087104  -0.45401776  1.5834119 ]
 [ 2.711011   -0.35716665 -1.3290031  -0.10792547  1.7408248 ]]


# `Unsharp`
----

In [3]:
import numpy as np
from numpy.typing import NDArray
from typing import TypeVar

F = TypeVar("F", float, np.floating)

def _unsharp(image: NDArray[F], A: F = 1.0) -> NDArray[F]:
    """
    Applies an unsharp mask to enhance edges in a grayscale image.

    Parameters:
    -----------
    image : NDArray[F]
        A 2D NumPy array representing the grayscale image.
    A : F, default=1.0
        The sharpening intensity factor (higher values enhance edges more).

    Returns:
    --------
    NDArray[F]
        The sharpened image after applying the unsharp mask.

    Notes:
    ------
    - The unsharp kernel is computed as:
      ```
      [[  0  -1   0 ]
       [ -1  A+4 -1 ]
       [  0  -1   0 ]] / A
      ```
    - Larger values of `A` increase edge contrast.
    - Uses convolution to apply the sharpening filter.
    - Clips output values to the valid image range [0, 255].

    Complexity:
    -----------
    - O(n * m * 3^2), where (n, m) are the image dimensions.
    """

    # Ensure A is not zero to avoid division errors
    if A == 0:
        raise ValueError("Sharpening factor 'A' must be nonzero.")

    # Define unsharp kernel
    unsharp_kernel = np.array([
        [0,  -1,  0],
        [-1, A + 4, -1],
        [0,  -1,  0]
    ], dtype = np.float32) / A

    # Apply convolution using the unsharp kernel
    unsharp_image = _convolve(image, unsharp_kernel)

    # Clip to valid image range and return
    return np.clip(unsharp_image, 0, 255, out = unsharp_image)


if __name__ == "__main__":
    # Create a sample 5x5 grayscale image (values from 0 to 255)
    image = np.random.randint(0, 256, (5, 5), dtype = np.uint8).astype(np.float32)

    # Apply unsharp masking with intensity factor A = 1.5
    sharpened_image = _unsharp(image, A = 1.5)

    print("Sharpened Image:\n", sharpened_image)



Sharpened Image:
 [[255.       255.       255.         0.       255.      ]
 [  0.       182.33331    0.       255.         0.      ]
 [255.        58.333313   0.       255.        91.33334 ]
 [255.       255.       255.         0.       255.      ]
 [ 67.66667    0.       255.         0.       255.      ]]


# `Sobel`
----

In [4]:
import numpy as np
from numpy.typing import NDArray
from typing import TypeVar, Literal

F = TypeVar("F", float, np.floating)

def _sobel(image: NDArray[F], axis: Literal["x", "y"] | None = None, div: F = 1.0) -> NDArray[F]:
    """
    Applies a Sobel filter to detect edges in a grayscale image.

    Parameters:
    -----------
    image : NDArray[F]
        A 2D NumPy array representing the grayscale image.
    axis : Literal["x", "y"], optional
        Specifies the direction of edge detection:
        - "x" applies the Sobel filter for horizontal edges.
        - "y" applies the Sobel filter for vertical edges.
        - None (default) applies both filters separately.
    div : F, default=1.0
        A divisor to scale the output intensity.

    Returns:
    --------
    NDArray[F]
        The filtered image with detected edges.

    Notes:
    ------
    - The Sobel operator enhances gradient edges by detecting intensity changes.
    - The "x" filter detects horizontal edges, and the "y" filter detects vertical edges.
    - If `axis` is None, it computes both x and y gradients and combines them as:
      ```
      sqrt(S_x^2 + S_y^2)
      ```
    - The result is scaled using `div` to control intensity.
    - Works only on grayscale images.

    Complexity:
    -----------
    - O(n * m * 3^2), where (n, m) are the image dimensions.
    """

    # Define Sobel kernel
    sobel_x = np.array([
        [-1,  0,  1],
        [-2,  0,  2],
        [-1,  0,  1]
    ], dtype = np.float32)

    sobel_y = sobel_x.T  # Transpose for vertical edge detection

    if axis == "x":
        filtered_image = _convolve(image, sobel_x)
    elif axis == "y":
        filtered_image = _convolve(image, sobel_y)
    elif axis is None:
        # Compute gradient magnitude from both axes
        grad_x = _convolve(image, sobel_x)
        grad_y = _convolve(image, sobel_y)
        filtered_image = np.sqrt(grad_x**2 + grad_y**2)
    else:
        raise ValueError("Invalid axis. Choose 'x', 'y', or None.")

    # Scale intensity and return
    return filtered_image / div


if __name__ == "__main__":
    # Create a sample 5x5 grayscale image
    image = np.random.rand(5, 5).astype(np.float32)

    # Apply Sobel filter for different edge detection modes
    sobel_x = _sobel(image, axis = "x")
    sobel_y = _sobel(image, axis = "y")
    sobel_combined = _sobel(image)

    print("Sobel X:\n", sobel_x)
    print("Sobel Y:\n", sobel_y)
    print("Combined Sobel:\n", sobel_combined)


Sobel X:
 [[ 2.3365278  -1.0241722  -1.5406154   0.50479174 -0.79591227]
 [ 2.047675    0.48153245 -1.3844892  -0.05224553 -0.66318583]
 [ 0.6495645   0.634223    0.15998489 -0.15126479 -0.8095494 ]
 [ 0.975396   -0.19784892  0.5307941   0.7894637  -1.5061901 ]
 [ 1.8278071  -0.10674536 -0.3008802   0.8807966  -1.5269268 ]]
Sobel Y:
 [[ 1.1981442   2.356407    2.4259539   1.879003    1.8175483 ]
 [-1.6141598  -1.5555707   0.03671326  0.18275306 -0.18010283]
 [-0.55748403 -1.8614931  -1.7566125  -0.4571975  -0.3397243 ]
 [ 0.67046714  1.4682357   0.9020926   0.91421515  1.0408297 ]
 [-0.64066017 -0.4949139  -0.66934144 -1.4218056  -1.477824  ]]
Combined Sobel:
 [[2.6258163  2.5693545  2.8738039  1.9456277  1.9841769 ]
 [2.6073904  1.628396   1.3849759  0.1900744  0.68720627]
 [0.85599214 1.9665694  1.7638829  0.48157093 0.8779424 ]
 [1.1836061  1.4815061  1.0466677  1.2079083  1.830829  ]
 [1.9368335  0.5062947  0.7338575  1.6725231  2.1249633 ]]


# `Gradient Magnitude`
---

In [5]:
import numpy as np
from numpy.typing import NDArray
from typing import TypeVar

F = TypeVar("F", bound = np.floating)

def _gradient(image: NDArray[F], div: F = 8.0) -> NDArray[F]:
    """
    Computes the gradient magnitude of an image using the Sobel operator.

    Parameters:
    -----------
    image : NDArray[F]
        A 2D NumPy array representing the grayscale image.
    div : F, default=8.0
        A divisor to scale the Sobel filter output.

    Returns:
    --------
    NDArray[F]
        The gradient magnitude of the image.
    """

    # Compute Sobel gradients
    g_x = _sobel(image, axis = "x", div = div)
    g_y = _sobel(image, axis = "y", div = div)

    # Compute gradient magnitude and return
    return np.sqrt(g_x**2 + g_y**2, out = g_x)  # Uses g_x as output to save memory

if __name__ == "__main__":
    # Create a sample 5x5 grayscale image
    image = np.random.rand(5, 5).astype(np.float32)

    # Compute gradient magnitude
    gradient = _gradient(image)

    print("Gradient Magnitude:\n", gradient)


Gradient Magnitude:
 [[0.17328542 0.17257163 0.2867684  0.38841727 0.26663983]
 [0.07472748 0.02503548 0.28623328 0.13830188 0.33584097]
 [0.10122409 0.04768483 0.34426916 0.07294346 0.40823177]
 [0.16199142 0.17306888 0.28127635 0.07938599 0.38254777]
 [0.13283534 0.20913847 0.38854763 0.40529132 0.41129723]]
